<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/4.%20%E7%94%BB%E5%83%8F%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google検索から画像をスクレイピングする

Google driveをマウントする

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


google_images_dowloadをインストール

In [4]:
!pip install google_images_download

     |████████████████████████████████| 911kB 11.2MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14550 sha256=f6f3a2a53cf18f1301a6515348e5a7a215e8fec72ea2b8cc00bf4f46a626cc4c
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


スクレイピングする

In [5]:
from google_images_download import google_images_download
response = google_images_download.googleimagesdownload()

response.download({
    "keywords": "イチゴ",
    "limit": 3,
    "no_numbering": True,
    "output_directory": "/content/drive/My Drive/AI_laboratory_course/images",
    "image_directory": "strawberry",
    "format": "jpg" or "jpeg",
    "print_urls": False,
    "chromedriver": "/content/drive/My Drive/AI_laboratory_course/chromedriver.exe"  #chromedriverをダウンロードして指定の場所に置いておく
    })

"""
response.download({
    "keywords": "リンゴ",
    "limit": 500,
    "no_numbering": True,
    "output_directory": "/content/drive/My Drive/AI_laboratory_course/images",
    "image_directory": "apple",
    "print_urls": False,
    "format": "jpg" or "jpeg",
     "chromedriver": "/content/drive/My Drive/AI_laboratory_course/chromedriver.exe"
    })
"""


Item no.: 1 --> Item name = \u30a4\u30c1\u30b4
Evaluating...
Starting Download...
Completed Image ====> 1.02215_2007_0315_IMG_1544.jpg
Completed Image ====> 2.c61aa8590d330141d2d914da5273b9cf_m-1400x933.jpg
Completed Image ====> 3.53S.jpg

Errors: 0



'\nresponse.download({\n    "keywords": "リンゴ",\n    "limit": 500,\n    "no_numbering": True,\n    "output_directory": "images",\n    "image_directory": "apple",\n    "print_urls": False,\n    "format": "jpg" or "jpeg",\n    "chromedriver": r"C:\\Users\\ykita\\python\\chromedriver.exe"\n    })\n'